In [0]:
!pip install flask-ngrok


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

path_1 = "/content/drive/My Drive/Colab Notebooks/planet_classification_w_localization/dnnmodel"

def load_model_x(path):
  model = load_model(os.path.join(path,'model.h5'))
  print("Model loading succesfully..")
  return model
model = load_model_x(path_1)

Model loading succesfully..


In [0]:
import os
import numpy as np

from flask import Flask, redirect, url_for, request, render_template,send_file
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

# Define a flask app
static_folder   ='/content/drive/My Drive/Colab Notebooks/planet_classification_w_localization/static'
template_folder = '/content/drive/My Drive/Colab Notebooks/planet_classification_w_localization/templates'
app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)
run_with_ngrok(app)   #starts ngrok when the app is run


def get_file_path_and_save(request):
    # Get the file from post request
    f = request.files['file']

    # Save the file to ./uploads
    basepath = "/content/drive/My Drive/Colab Notebooks/planet_classification_w_localization"
    if not os.path.exists(os.path.join(basepath,'uploads')):
        os.mkdir(os.path.join(basepath,'uploads'))
    file_path = os.path.join(
        basepath, 'uploads', secure_filename(f.filename))
    f.save(file_path)
    return file_path,secure_filename(f.filename)



@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

@app.route('/images/<filename>', methods=['GET'])
def images(filename):
    return send_file(os.path.join('/content/drive/My Drive/Colab Notebooks/planet_classification_w_localization/uploads', filename))


@app.route('/predictdnnmodel', methods=['GET', 'POST'])
def predictResNet50():
    if request.method == 'POST':
        file_path,name= get_file_path_and_save(request)
        img = cv2.imread(file_path)
        img = cv2.resize(img,(300,300))
        k,l = model.predict(np.array([img]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        pred_labels = []
        for i in range(len(k)):
            r = np.round(k[i],3).argmax()
            if r ==0 : pred_labels.append("EARTH")
            elif r ==1: pred_labels.append("JUPITER")
            elif r ==2: pred_labels.append("SATURN")
            elif r ==3: pred_labels.append("MARS")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        m = 0 
        img = cv2.rectangle(img, (l[m-1][0], l[m-1][1]), (l[m-1][2], l[m-1][3]), (0,255,0), 2)
        img = cv2.imwrite(file_path,img)

        result = pred_labels[0]               # Convert to string
        return result
    return None


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d7adf1b022de.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jun/2020 18:09:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:14] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:14] "GET /static/js/resNet50.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:42] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:09:42] "GET /static/js/resNet50.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:12:38] "POST /predictResNet50 HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:12:39] "GET /images/2Q___8.jpg HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:14:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:15:24] "POST /predictResNet50 HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:15:31] "POST /predictResNet50 HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:15:31] "GET /images/images_30.jpg HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 18:15:39] "POST /predictResN